In [10]:
import os
from os.path import basename, dirname, join, exists
import sys

import numpy as np
import numpy.ma as ma
import scipy.stats as stats
import pandas as pd
import geopandas as gpd

from scipy.optimize import minimize

# from datetime import datetime
# from datetime import timedelta
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as lines

# from pandas.tseries import converter

In [2]:
doc_dir = os.getcwd()
while basename(doc_dir) != 'Documents':
    doc_dir = dirname(doc_dir)
    
# dir of all gwfm data
gwfm_dir = join(dirname(doc_dir),'Box/research_cosumnes/GWFlowModel')

flopy_dir = doc_dir+'/GitHub/flopy'
if flopy_dir not in sys.path:
    sys.path.insert(0, flopy_dir)
import flopy 
import flopy.utils.binaryfile as bf


In [3]:
# set box directory for output figures and data
box_dir = gwfm_dir+'/Levee_setback/levee_setback_distance_analysis/'

# tprogs_id = '' # original tprogs with conditioning data in output tsim
# tprogs_id = '_no_conditioning'
tprogs_id = '_no_cond_c3d'


data_dir = box_dir+ tprogs_id+'/data_output/'
fig_dir = box_dir+tprogs_id+'/figures/'

chan_dir = box_dir+'channel_data/'
gis_dir = chan_dir+'GIS/'

In [7]:
nrow = 100
ncol = 230

In [5]:
setbacks = np.arange(0, 3400,200)
# smoothed XS data used for setback analysis
xs_levee_smooth = pd.read_csv(chan_dir+'xs_levee_smooth.csv', index_col='dist_from_right_m')
num_segs = xs_levee_smooth.shape[1]

# load array identifying row,col to XS id (1,28)
xs_arr = np.loadtxt(chan_dir+'XS_num_grid_reference.tsv')

# load flood typology characteristics (based on daily data 1908 - 2014) - median values 
#"cms_pk" for peak discharge, "pk_loc" for time to peak, and "log_no_d" for duration
flood_type = pd.read_csv(join(box_dir, 'whipple_grp6_w97ftmedians.csv'),index_col='Group.1')


In [11]:
tic = time.time()
ft_in=2
region = 'local_1'
# region = 'regional'
T_in = int(10**flood_type.loc[ft_in,'log_no_d'])
p_l_in = flood_type.loc[ft_in,'pk_loc']
tp_in = int(p_l_in*T_in)
rch_hf_all = np.zeros((100, len(setbacks),nrow,ncol))
Q_all = np.zeros((100, T_in, len(setbacks),xs_levee_smooth.shape[1]+1))

# filter out for only those realizations that successfully ran
base_fn = join(data_dir, region, 'type'+str(ft_in))
r_out = pd.Series(os.listdir(base_fn)).str.extract(r'(\d{3})')[0].unique().astype(int)
# takes a 
for t in r_out: # np.arange(0,100): #[0]:
    r_fn = join(base_fn,'r'+str(t).zfill(3)+'_')
    # saving all of the flow at all steps, setbacks is needed to post-process
    Q_in = np.loadtxt(r_fn+'flow.tsv')
    Q = np.reshape(Q_in, ((T_in, len(setbacks), xs_levee_smooth.shape[1]+1)))
    Q_all[t,:] = np.copy(Q)
    
    # for recharge we want to aggregate across time steps but look at differences across setbacks
    rch_in = np.loadtxt(r_fn+'recharge.tsv')
    rch_sum = np.reshape(rch_in, (len(setbacks), nrow, ncol))
    rch_hf_all[t] = np.copy(rch_sum)
# convert to m3/day and will have the total recharged after summing individual days
rch_hf_all = rch_hf_all*86400

# convert to total AF from total m3
# rch_hf_all = rch_hf_all/((0.3048**3)*43560)

toc = time.time()
print((toc-tic))

75.78515219688416


In [ ]:

fig, ax = plt.subplots(figsize=(6,3), dpi=300)

# convert from 
rch_xs_sum = pd.DataFrame(np.sum(rch_hf_all, axis=(2,3)), columns= setbacks).transpose()/1E6
rch_xs_sum.plot(legend=False, color='lightgray', ax =ax)
rch_xs_sum.mean(axis=1).plot(color='black', ax=ax)


ax.set_xlabel('Setback Distance (m)')
ax.set_ylabel('Total Recharge ($10^6$ $m^3$)')

# 1d so the smoothing is specific to each realization
from scipy.ndimage import gaussian_filter1d
# smooth across setback distances so the changes in high flow area are less abrupt, makes it easier to find gradients
smooth = gaussian_filter1d(rch_xs_sum,2, axis=0)

smooth = pd.DataFrame(smooth, index= setbacks)

fig, ax = plt.subplots(figsize=(6,3), dpi=300)

smooth.plot(legend=False, color='lightgray', ax=ax, label='Realizations')
smooth.mean(axis=1).plot(color='black', ax=ax, label='Mean')

ax.set_title('Gaussian Filter 1D (2 sigma)')

ax.set_xlabel('Setback Distance (m)')

# axes[1].legend()
# tranpose to fit format for gradient analysis used previously
smooth = smooth.transpose()


In [ ]:
r = np.arange(0,100)
# can look at maximum of first derivative, or look where second derivative goes from + to -
grad1_df = pd.DataFrame(np.gradient(smooth)[1],  columns = setbacks)

grad2_df = pd.DataFrame(np.gradient(grad1_df)[1],  columns = setbacks)

In [ ]:
# np.sign gives + or -, np diff gives out[i] = a[i+1] - a[i]. 
# + to - means max of 1st derivative. So diff ==-2 means max, but to account for difference go up one indice
fig,ax = plt.subplots(figsize=(6,3), dpi=300)
max_df = pd.DataFrame(np.diff(np.sign(grad2_df)), columns = setbacks[:-1])
max_df = 1*(max_df==-2)
max_df.loc[r_out].transpose().plot(kind='bar',legend=False,stacked=True,ax=ax, color='lightgray') #.loc[n]
plt.ylabel('Maximum gradient count\n by realization')
plt.xlabel('Setback distance (m)')

In [ ]:
# t=0

fig,ax = plt.subplots(figsize=(6,3), dpi=300)
Q_plt = pd.DataFrame(Q_all[:,tp,-1,:], columns = np.arange(0,29*2,2)).transpose()
Q_plt.plot(color='lightgray', legend=False, ax=ax)
# plot min, max
Q_plt.loc[:,Q_plt.mean(axis=0).isin(Q_plt.mean(axis=0).quantile([0, 1]).values)].plot(legend=False, color='red', ax=ax)

Q_plt.mean(axis=1).plot(color='black', label='Averaged Realizations', ax=ax)
plt.xlabel('Distance downstream (km)')
plt.ylabel('Discharge ($m^3/s$)')

fig.tight_layout()

In [ ]:
max_s = max_df.loc[r_out].sum().argmax()
Q_s = Q_all[:,tp,max_s,:].mean(axis=0)
d_Q_s = Q_s[0]-Q_s[-1]

# summarize optimal setback distance with mean recharge and mean flow reduction
print(flood_type.loc[ft_in,'Typology'], setbacks[max_s], '%.0f' %rch_xs_sum.iloc[max_s].mean(),'%.0f' %d_Q_s)